# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 28 2022 9:35AM,253894,10,0086271104,ISDIN Corporation,Released
1,Dec 28 2022 9:35AM,253894,10,0086271107,ISDIN Corporation,Released
2,Dec 28 2022 9:35AM,253894,10,0086271106,ISDIN Corporation,Released
3,Dec 28 2022 9:35AM,253894,10,0086271105,ISDIN Corporation,Released
4,Dec 28 2022 9:35AM,253894,10,0086271108,ISDIN Corporation,Released
5,Dec 28 2022 9:35AM,253894,10,0086271123,ISDIN Corporation,Released
6,Dec 28 2022 9:35AM,253894,10,0086271122,ISDIN Corporation,Released
7,Dec 28 2022 9:35AM,253894,10,0086271125,ISDIN Corporation,Released
8,Dec 28 2022 9:35AM,253894,10,0086271119,ISDIN Corporation,Released
9,Dec 28 2022 9:35AM,253894,10,0086271118,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,253894,Released,75
36,253895,Released,1
37,253896,Released,1
38,253897,Released,26
39,253898,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
253894,NaN,75.0
253895,NaN,1.0
253896,NaN,1.0
253897,NaN,26.0
253898,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
253800,0.0,64.0
253807,0.0,12.0
253811,0.0,77.0
253813,0.0,66.0
253814,0.0,53.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
253800,0,64
253807,0,12
253811,0,77
253813,0,66
253814,0,53


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,253800,0,64
1,253807,0,12
2,253811,0,77
3,253813,0,66
4,253814,0,53


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,253800,,64
1,253807,,12
2,253811,,77
3,253813,,66
4,253814,,53


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 28 2022 9:35AM,253894,10,ISDIN Corporation
75,Dec 28 2022 9:20AM,253897,10,ISDIN Corporation
101,Dec 28 2022 9:18AM,253898,10,"Methapharm, Inc."
107,Dec 28 2022 9:16AM,253891,10,ISDIN Corporation
111,Dec 28 2022 9:14AM,253895,10,"Lupin Research, Inc."
112,Dec 28 2022 9:11AM,253896,10,Emerginnova
113,Dec 28 2022 9:10AM,253892,12,"SD Head USA, LLC"
114,Dec 28 2022 8:59AM,253890,10,ISDIN Corporation
128,Dec 28 2022 8:42AM,253884,10,"Nextsource Biotechnology, LLC"
129,Dec 28 2022 8:41AM,253885,19,Else Nutrition


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Dec 28 2022 9:35AM,253894,10,ISDIN Corporation,,75
1,Dec 28 2022 9:20AM,253897,10,ISDIN Corporation,,26
2,Dec 28 2022 9:18AM,253898,10,"Methapharm, Inc.",,6
3,Dec 28 2022 9:16AM,253891,10,ISDIN Corporation,,4
4,Dec 28 2022 9:14AM,253895,10,"Lupin Research, Inc.",,1
5,Dec 28 2022 9:11AM,253896,10,Emerginnova,,1
6,Dec 28 2022 9:10AM,253892,12,"SD Head USA, LLC",1,
7,Dec 28 2022 8:59AM,253890,10,ISDIN Corporation,,14
8,Dec 28 2022 8:42AM,253884,10,"Nextsource Biotechnology, LLC",,1
9,Dec 28 2022 8:41AM,253885,19,Else Nutrition,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 28 2022 9:35AM,253894,10,ISDIN Corporation,75,
1,Dec 28 2022 9:20AM,253897,10,ISDIN Corporation,26,
2,Dec 28 2022 9:18AM,253898,10,"Methapharm, Inc.",6,
3,Dec 28 2022 9:16AM,253891,10,ISDIN Corporation,4,
4,Dec 28 2022 9:14AM,253895,10,"Lupin Research, Inc.",1,
5,Dec 28 2022 9:11AM,253896,10,Emerginnova,1,
6,Dec 28 2022 9:10AM,253892,12,"SD Head USA, LLC",,1
7,Dec 28 2022 8:59AM,253890,10,ISDIN Corporation,14,
8,Dec 28 2022 8:42AM,253884,10,"Nextsource Biotechnology, LLC",1,
9,Dec 28 2022 8:41AM,253885,19,Else Nutrition,1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 28 2022 9:35AM,253894,10,ISDIN Corporation,75,
1,Dec 28 2022 9:20AM,253897,10,ISDIN Corporation,26,
2,Dec 28 2022 9:18AM,253898,10,"Methapharm, Inc.",6,
3,Dec 28 2022 9:16AM,253891,10,ISDIN Corporation,4,
4,Dec 28 2022 9:14AM,253895,10,"Lupin Research, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Dec 28 2022 9:35AM,253894,10,ISDIN Corporation,75.0,NaN
1,Dec 28 2022 9:20AM,253897,10,ISDIN Corporation,26.0,NaN
2,Dec 28 2022 9:18AM,253898,10,"Methapharm, Inc.",6.0,NaN
3,Dec 28 2022 9:16AM,253891,10,ISDIN Corporation,4.0,NaN
4,Dec 28 2022 9:14AM,253895,10,"Lupin Research, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Dec 28 2022 9:35AM,253894,10,ISDIN Corporation,75.0,0.0
1,Dec 28 2022 9:20AM,253897,10,ISDIN Corporation,26.0,0.0
2,Dec 28 2022 9:18AM,253898,10,"Methapharm, Inc.",6.0,0.0
3,Dec 28 2022 9:16AM,253891,10,ISDIN Corporation,4.0,0.0
4,Dec 28 2022 9:14AM,253895,10,"Lupin Research, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,7361733,571.0,1.0
12,253892,0.0,1.0
19,2030944,34.0,7.0
22,253857,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,7361733,571.0,1.0
1,12,253892,0.0,1.0
2,19,2030944,34.0,7.0
3,22,253857,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,571.0,1.0
1,12,0.0,1.0
2,19,34.0,7.0
3,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,571.0
1,12,Released,0.0
2,19,Released,34.0
3,22,Released,1.0
4,10,Executing,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19,22
Status,,,,
Executing,1.0,1.0,7.0,0.0
Released,571.0,0.0,34.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19,22
0,Executing,1.0,1.0,7.0,0.0
1,Released,571.0,0.0,34.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19,22
0,Executing,1.0,1.0,7.0,0.0
1,Released,571.0,0.0,34.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()